In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
import transformers 
transformers.__version__
import trl
trl.__version__

C:\Users\rabhishek1\gemma2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'0.9.4'

In [2]:
template_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [3]:
def format_prompt(example):
    chat = example["messages"]
    prompt = template_tokenizer.apply_chat_template(chat, tokenize=False)
    return {"text":prompt}

In [4]:
dataset = (load_dataset("HuggingFaceH4/ultrachat_200k",split="test_sft").shuffle(seed=42).select(range(3_000)))

In [5]:
dataset = dataset.map(format_prompt)

In [6]:
dataset["text"][2576]

'<|user|>\nGiven the text: Knock, knock. Who’s there? Hike.\nCan you continue the joke based on the given text material "Knock, knock. Who’s there? Hike"?</s>\n<|assistant|>\nSure! Knock, knock. Who\'s there? Hike. Hike who? Hike up your pants, it\'s cold outside!</s>\n<|user|>\nCan you tell me another knock-knock joke based on the same text material "Knock, knock. Who\'s there? Hike"?</s>\n<|assistant|>\nOf course! Knock, knock. Who\'s there? Hike. Hike who? Hike your way over here and let\'s go for a walk!</s>\n'

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [8]:
model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
model = AutoModelForCausalLM.from_pretrained(model_name,device_map="auto")

In [9]:
model.config.use_cache = False
model.config.pretraining_tp = 1

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token="<PAD>"
tokenizer.padding_side="left"

In [11]:
from peft import LoraConfig, get_peft_model

In [12]:
peft_config = LoraConfig(
    lora_alpha = 32,
    lora_dropout =0.1,
    r =64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules = ["k_proj","gate_proj","v_proj","up_proj","q_proj","o_proj","down_proj"]
)

In [13]:
#Prepare model for training
model = get_peft_model(model, peft_config)

ModuleNotFoundError: No module named 'triton.ops'

In [ ]:
from transformers import TrainingArguments
output_dir = "./results"
training_arguments = TrainingArguments(
    output_dir = output_dir,
    per_device_train_batch_size =2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    num_train_epochs=1,
    logging_steps=10,
    gradient_checkpointing=True)
    

In [ ]:
from trl import SFTTrainer
model.config.label_names = ["labels"]
trainer = SFTTrainer(model=model,
                     train_dataset=dataset,
                     dataset_text_field="text",
                    tokenizer = tokenizer,
                    args = training_arguments,
                    max_seq_length=512,
                    peft_config = peft_config)

In [ ]:
trainer.train()
trainer.model.save_pretrained("TinyLlama-1.1B-lora")

In [ ]:
from peft import AutoPeftModelForCausalLM

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained("TinyLlama-1.1B-lora",low_cpu_mem_usage=True,device_map="auto")
merged_model = model.merge_and_unload()


In [ ]:
from transformers import pipeline
prompt="""<|user|>
Tell me something about Large Language Models.</s>
<|assistant|>
"""
pipe = pipeline(task="text_generation",model=merged_model,tokenizer=tokenizer)
print(pipe(prompt)[0]["generated_text"])

In [14]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model

# Load tokenizer and model
model_name = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# Apply LoRA
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)
model = get_peft_model(model, peft_config)

# Load and format dataset
dataset = load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft")
dataset = dataset.shuffle(seed=42).select(range(3000))

def format_prompt(example):
    messages = example["messages"]
    prompt = ""
    for msg in messages:
        role = msg["role"]
        content = msg["content"]
        if role == "user":
            prompt += f"User: {content}\n"
        elif role == "assistant":
            prompt += f"Assistant: {content}\n"
    return {"text": prompt.strip()}

formatted_dataset = dataset.map(format_prompt)

# Tokenize and prepare labels
def tokenize(example):
    tokenized = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = formatted_dataset.map(tokenize, batched=True)

# Set label_names to avoid warning
model.config.label_names = ["labels"]

# Training configuration
training_args = SFTConfig(
    output_dir="./sft_gemma3b_it_lora",
    per_device_train_batch_size=4,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=500,
    max_seq_length=512,
    packing=False,
    report_to=[]
)

# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args
)

# Start training
trainer.train()

# Save the fine-tuned model
trainer.model.save_pretrained("gemma3b_it_lora_trained")


C:\Users\rabhishek1\gemma2\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\LLM_NER\HF_MODEL\hub\models--google--gemma-3-1b-it. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Exception: data did not match any variant of untagged enum ModelWrapper at line 2379610 column 3